In [1]:
import numpy as np
import open3d as o3d

In [4]:
def remove_rover(pcd):
    """Remove rover vehicle from the point cloud

    Args:
        pcd ([type]): [description]

    Returns:
        [type]: [description]
    """
    coords = np.asarray(pcd.points)

    rover_width = 2 # x, 2
    rover_length = 5 # y, 5
    rover_height = 1.8 # z, 3.6

    # Assuming sensor is the origin and is on top of the vehicle
    coords = np.delete(coords, np.where((coords[:,0] > -rover_width / 2) & (coords[:,0] < rover_width / 2) & (coords[:,1] > -rover_length / 2) & (coords[:,1] < rover_length / 2) & (coords[:,2] < rover_height)), axis=0)

    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(coords)

    return new_pc


def remove_ground(pcd):
    n_scan = 32
    horizon_scan = 1800
    # ang_top = 10.67 # HDL-32E
    # ang_bottom = -30.67 # HDL-32E
    ang_top = 15 # VLP-32C
    ang_bottom = -25 # VLP-32C
    ang_res_x = 360/horizon_scan
    ang_res_y = (ang_top-ang_bottom) / (n_scan - 1)
    ground_scan_ind = 20
    sensor_mount_angle = 0
    sensor_minimum_range = 1

    coords = np.asarray(pcd.points)

    # Reorder points
    total_pts = coords.shape[0]
    reordered_coords = np.zeros(shape=[total_pts, 4])
    for [x, y, z] in coords:
        if np.linalg.norm([x, y, z]) < sensor_minimum_range:
            continue

        vertical_angle = np.arctan2(z, np.linalg.norm([x, y])) * 180.0 / np.pi

        if vertical_angle < ang_bottom or vertical_angle > ang_top:
            continue

        vert_id = np.round((vertical_angle - ang_bottom) / ang_res_y)

        horizon_angle = np.arctan2(y, x) * 180.0 / np.pi

        if horizon_angle < 0:
            horizon_angle += 360

        horizon_id = np.round(horizon_angle / ang_res_x)

        index = int(horizon_id + vert_id * horizon_scan)

        if index >= total_pts:
            continue

        reordered_coords[index, :3] = [x, y, z]

    # Ground removal
    if horizon_scan * ground_scan_ind >= total_pts:
        threshold = total_pts
    else:
        threshold = horizon_scan * ground_scan_ind

    for i in range(threshold):
        lower_id = i
        upper_id = i + horizon_scan

        if upper_id >= reordered_coords.shape[0]:
            continue

        diffX = reordered_coords[upper_id, 0] - reordered_coords[lower_id, 0]
        diffY = reordered_coords[upper_id, 1] - reordered_coords[lower_id, 1]
        diffZ = reordered_coords[upper_id, 2] - reordered_coords[lower_id, 2]

        angle = np.arctan2(diffZ, np.linalg.norm([diffX, diffY])) * 180.0 / np.pi

        if np.abs(angle - sensor_mount_angle) <= 10:
            reordered_coords[lower_id, 3] = 1
            reordered_coords[upper_id, 3] = 1

    reordered_coords = reordered_coords[np.logical_not(np.logical_and(reordered_coords[:, 0] == 0, reordered_coords[:, 1] == 0, reordered_coords[:, 2] == 0))]
    
    new_points = reordered_coords[:, :3][reordered_coords[:, -1] != 1]

    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(new_points)

    return new_pc


def read_pcd(path, downsample=0, non_rover=True, non_ground=False):
    """Read point cloud

    Args:
        path ([string]): path to the point cloud

    Returns:
        [type]: [description]
    """
    pcd = o3d.io.read_point_cloud(path)

    if downsample > 0:
        pcd = pcd.voxel_down_sample(voxel_size)
    
    if non_rover:
        pcd = remove_rover(pcd)

    if non_ground:
        pcd = remove_ground(pcd)

    return pcd

In [7]:
pc = read_pcd('/mnt/G/LiDAR datasets/UrbanNav/HK_20190428/velodyne/1556456513.131748000.pcd', 0, True, True)
np.savetxt('/mnt/G/LiDAR datasets/UrbanNav/HK_20190428/demo/1556456513.131748000.csv', pc.points, delimiter=',')

In [8]:
pc = read_pcd('/mnt/G/LiDAR datasets/UrbanNav/HK_20190428/velodyne/1556456514.132696000.pcd', 0, True, True)
np.savetxt('/mnt/G/LiDAR datasets/UrbanNav/HK_20190428/demo/1556456514.132696000.csv', pc.points, delimiter=',')